In [1]:
import glob
import pandas as pd
from autogluon.tabular import TabularDataset, TabularPredictor
from autogluon.common.features.feature_metadata import FeatureMetadata

In [2]:
training_files = 5
data_files = glob.glob('bid_data/*.tsv')
dfs = [pd.read_csv(file, sep='\t') for file in data_files[:training_files]]

train_data = TabularDataset(pd.concat(dfs, ignore_index=True))

In [3]:
train_data.head()

,bid_pos,go_alone,def_alone,def_pos_rel,turn_card_level,bid_turn_suit,bid_next_suit,bid_green_suit,top_trump_strg,top_2_trump_strg,top_3_trump_strg,num_trump,num_next,num_voids,num_singletons,num_off_aces,num_tricks,points
0,0,0,0,0,6,1,0,0,7,7,7,2,0,1,1,0,2,-2
1,1,1,0,0,6,1,0,0,0,0,0,0,1,1,1,2,3,1
2,2,1,0,0,6,1,0,0,0,0,0,0,2,1,1,1,0,-2
3,1,1,1,1,6,1,0,0,0,0,0,0,2,1,1,1,3,4
4,2,1,1,1,6,1,0,0,10,13,13,3,1,1,2,0,4,4


In [4]:
label = 'points'
train_data[label].describe()

count    759360.000000
mean          0.143026
std           2.625323
min          -4.000000
25%          -2.000000
50%          -1.000000
75%           4.000000
max           4.000000
Name: points, dtype: float64

In [5]:
exclude = []
outputs = ['num_tricks', 'points']
outputs.remove(label)

type_map = {k: 'int' for k in list(train_data)}
type_map['bid_pos'] = 'category'
type_map[label] = 'float'
metadata = FeatureMetadata(type_map_raw=type_map)

predictor = TabularPredictor(problem_type='regression', label=label)
predictor.fit(train_data.drop(columns=(exclude + outputs)), feature_metadata=metadata)

No path specified. Models will be saved in: "AutogluonModels/ag-20240904_044315"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.11.9
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #202405300957~1722440358~24.04~aa0a43f~dev-Ubuntu SMP PREEMPT_DY
CPU Count:          8
Memory Avail:       6.37 GB / 15.34 GB (41.5%)
Disk Space Avail:   125.27 GB / 452.95 GB (27.7%)
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets.
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='best_quality'   : Maximize accuracy. Default time_limit=3600.
	presets='high_quality'   : Strong accuracy with fast inference speed. Default time_limit=3600.
	presets='good_quality'   : Good accuracy with very fast inference speed. Default time_limit=3600.
	presets='medium_

[1000]	valid_set's rmse: 1.95088


	-1.9506	 = Validation score   (-root_mean_squared_error)
	21.72s	 = Training   runtime
	0.16s	 = Validation runtime
Fitting model: LightGBM ...


[1000]	valid_set's rmse: 1.94786


	-1.9476	 = Validation score   (-root_mean_squared_error)
	20.19s	 = Training   runtime
	0.15s	 = Validation runtime
Fitting model: RandomForestMSE ...
	-1.983	 = Validation score   (-root_mean_squared_error)
	248.47s	 = Training   runtime
	0.15s	 = Validation runtime
Fitting model: CatBoost ...
	-1.9482	 = Validation score   (-root_mean_squared_error)
	1241.85s	 = Training   runtime
	0.1s	 = Validation runtime
Fitting model: ExtraTreesMSE ...
	-1.9751	 = Validation score   (-root_mean_squared_error)
	211.29s	 = Training   runtime
	0.14s	 = Validation runtime
Fitting model: NeuralNetFastAI ...
	-1.9487	 = Validation score   (-root_mean_squared_error)
	418.36s	 = Training   runtime
	0.06s	 = Validation runtime
Fitting model: XGBoost ...
	-1.9484	 = Validation score   (-root_mean_squared_error)
	47.84s	 = Training   runtime
	0.17s	 = Validation runtime
Fitting model: NeuralNetTorch ...
	-2.2311	 = Validation score   (-root_mean_squared_error)
	290.66s	 = Training   runtime
	0.02s	 = Vali

In [6]:
test_data = TabularDataset(data_files[training_files])

y_pred = predictor.predict(test_data.drop(columns=([label] + exclude + outputs)))
y_pred.head()

Loaded data from: bid_data/bid_model_2-20240903_113350.tsv | Columns = 18 / 18 | Rows = 151660 -> 151660


0   -1.126498
1   -1.948905
2    0.777173
3   -0.788740
4    2.605490
Name: points, dtype: float32

In [7]:
predictor.evaluate(test_data, silent=True)

{'root_mean_squared_error': -1.9638647729226346,
 'mean_squared_error': -3.856764846326471,
 'mean_absolute_error': -1.4838659628737452,
 'r2': 0.4415199456831176,
 'pearsonr': 0.6644940281318878,
 'median_absolute_error': -1.171911358833313}

In [8]:
predictor.leaderboard(test_data)

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L2,-1.963865,-1.946495,root_mean_squared_error,19.006484,0.902292,1977.490214,0.008730,0.000615,0.039381,2,True,12
1,LightGBMLarge,-1.964474,-1.947506,root_mean_squared_error,2.271941,0.111979,16.190476,2.271941,0.111979,16.190476,1,True,11
2,LightGBM,-1.964944,-1.947555,root_mean_squared_error,4.069665,0.154104,20.192999,4.069665,0.154104,20.192999,1,True,4
3,CatBoost,-1.965141,-1.948244,root_mean_squared_error,1.321851,0.104484,1241.852211,1.321851,0.104484,1241.852211,1,True,6
4,NeuralNetFastAI,-1.965384,-1.948717,root_mean_squared_error,1.469038,0.063104,418.361262,1.469038,0.063104,418.361262,1,True,8
5,XGBoost,-1.966239,-1.948404,root_mean_squared_error,3.280546,0.174539,47.841382,3.280546,0.174539,47.841382,1,True,9
6,LightGBMXT,-1.968957,-1.950636,root_mean_squared_error,4.137777,0.155900,21.723251,4.137777,0.155900,21.723251,1,True,3
7,ExtraTreesMSE,-1.992916,-1.975087,root_mean_squared_error,2.446936,0.137568,211.289253,2.446936,0.137568,211.289253,1,True,7
8,RandomForestMSE,-2.000658,-1.983023,root_mean_squared_error,2.920244,0.150695,248.467985,2.920244,0.150695,248.467985,1,True,5
9,NeuralNetTorch,-2.213406,-2.231103,root_mean_squared_error,0.409628,0.022825,290.656370,0.409628,0.022825,290.656370,1,True,10
